In [1]:
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import mean_squared_error
import random


In [2]:

df = pd.read_csv("phoneprices.csv")

df

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,794,1,0.5,1,0,1,2,0.8,106,6,...,1222,1890,668,13,4,19,1,1,0,0
1996,1965,1,2.6,1,0,0,39,0.2,187,4,...,915,1965,2032,11,10,16,1,1,1,2
1997,1911,0,0.9,1,1,1,36,0.7,108,8,...,868,1632,3057,9,1,5,1,1,0,3
1998,1512,0,0.9,0,4,1,46,0.1,145,5,...,336,670,869,18,10,19,1,1,1,0


In [3]:

x = df.iloc[:,:-1]

x_normalized = (x - x.min()) / (x.max() - x.min())

x_normalized

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi
0,0.227789,0.0,0.68,0.0,0.052632,0.0,0.080645,0.555556,0.900000,0.142857,0.10,0.010204,0.170895,0.612774,0.285714,0.388889,0.944444,0.0,0.0,1.0
1,0.347361,1.0,0.00,1.0,0.000000,1.0,0.822581,0.666667,0.466667,0.285714,0.30,0.461735,0.993324,0.634687,0.857143,0.166667,0.277778,1.0,1.0,0.0
2,0.041416,1.0,0.00,1.0,0.105263,1.0,0.629032,0.888889,0.541667,0.571429,0.30,0.644388,0.811749,0.627205,0.428571,0.111111,0.388889,1.0,1.0,0.0
3,0.076152,1.0,0.80,0.0,0.000000,0.0,0.129032,0.777778,0.425000,0.714286,0.45,0.620408,0.858478,0.671566,0.785714,0.444444,0.500000,1.0,0.0,0.0
4,0.881764,1.0,0.28,0.0,0.684211,1.0,0.677419,0.555556,0.508333,0.142857,0.70,0.616327,0.475300,0.308658,0.214286,0.111111,0.722222,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.195725,1.0,0.00,1.0,0.000000,1.0,0.000000,0.777778,0.216667,0.714286,0.70,0.623469,0.927904,0.110102,0.571429,0.222222,0.944444,1.0,1.0,0.0
1996,0.977956,1.0,0.84,1.0,0.000000,0.0,0.596774,0.111111,0.891667,0.428571,0.15,0.466837,0.977971,0.474613,0.428571,0.555556,0.777778,1.0,1.0,1.0
1997,0.941884,0.0,0.16,1.0,0.052632,1.0,0.548387,0.666667,0.233333,1.000000,0.15,0.442857,0.755674,0.748530,0.285714,0.055556,0.166667,1.0,1.0,0.0
1998,0.675351,0.0,0.16,0.0,0.210526,1.0,0.709677,0.000000,0.541667,0.571429,0.25,0.171429,0.113485,0.163816,0.928571,0.555556,0.944444,1.0,1.0,1.0


In [4]:
y = pd.get_dummies(df.iloc[:,-1], prefix='y')

y

,y_0,y_1,y_2,y_3
0,0,1,0,0
1,0,0,1,0
2,0,0,1,0
3,0,0,1,0
4,0,1,0,0
...,...,...,...,...
1995,1,0,0,0
1996,0,0,1,0
1997,0,0,0,1
1998,1,0,0,0


In [5]:
x_train, x_test, y_train, y_test = train_test_split(x_normalized, y, test_size=0.3)

len_input = len(x_train.columns)
len_output = len(y_train.columns)

print("Len Input: {}".format(len_input))
print("Len Output: {}".format(len_output))

Len Input: 20
Len Output: 4


In [6]:
def objective_function(x, y, x_validation, y_validation, params):
    model = MLPClassifier(
        random_state=1, 
        max_iter=1000,
        alpha=params['alpha'],
        learning_rate_init=params['learning_rate']
    )

    model.fit(x, y)

    predictions = model.predict(x_validation)

    return mean_squared_error(predictions, y_validation)

In [7]:
grid_parameters = [
    { 'alpha': 0.002, 'learning_rate': 0.01 },
    { 'alpha': 0.003, 'learning_rate': 0.009 },
    { 'alpha': 0.004, 'learning_rate': 0.008 },
    { 'alpha': 0.005, 'learning_rate': 0.007 },
    { 'alpha': 0.006, 'learning_rate': 0.006 },
    { 'alpha': 0.007, 'learning_rate': 0.005 },
    { 'alpha': 0.008, 'learning_rate': 0.004 },
    { 'alpha': 0.009, 'learning_rate': 0.003 }
]

In [8]:
best_params = None
current_loss = 1000

for params in grid_parameters:
    loss = objective_function(x_train.values, y_train.values, x_test.values, y_test.values, params)
    print("Loss: {}".format(loss))

    if loss < current_loss:
        best_params = params
        loss = current_loss

print("Best params: {}".format(best_params))

Loss: 0.030833333333333334
Loss: 0.035
Loss: 0.03625
Loss: 0.03958333333333333
Loss: 0.043333333333333335
Loss: 0.03791666666666667
Loss: 0.043750000000000004
Loss: 0.03916666666666666
Best params: {'alpha': 0.009, 'learning_rate': 0.003}


In [9]:
num_samples = 10
best_params = None
current_loss = 1000

for i in range(num_samples):
    params = {
        'alpha': random.randrange(0, 100) * 0.0001,
        'learning_rate': random.randrange(0, 100) * 0.0001
    }

    loss = objective_function(x_train.values, y_train.values, x_test.values, y_test.values, params)
    print("Loss: {}".format(loss))

    if loss < current_loss:
        best_params = params
        loss = current_loss

print("Best params: {}".format(best_params))

/home/ralampay/workspace/pattern-recognition-course/env/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


Loss: 0.034583333333333334
Loss: 0.04166666666666667
Loss: 0.042083333333333334
Loss: 0.044583333333333336
Loss: 0.03625
Loss: 0.037083333333333336


/home/ralampay/workspace/pattern-recognition-course/env/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


Loss: 0.042083333333333334
Loss: 0.03708333333333334
Loss: 0.04666666666666667
Loss: 0.04041666666666667
Best params: {'alpha': 0.0066, 'learning_rate': 0.0017000000000000001}


/home/ralampay/workspace/pattern-recognition-course/env/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


In [10]:
x_samples = []
y_samples = []

num_samples = 10
for i in range(num_samples):
    params = {
        'alpha': random.randrange(0, 100) * 0.0001,
        'learning_rate': random.randrange(0, 100) * 0.0001
    }

    loss = objective_function(x_train.values, y_train.values, x_test.values, y_test.values, params)
    print("Loss: {}".format(loss))

    x1 = params['alpha']
    x2 = params['learning_rate']
    y = loss

    x_samples.append([x1, x2])
    y_samples.append([y])

Loss: 0.04375
Loss: 0.04791666666666666


/home/ralampay/workspace/pattern-recognition-course/env/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


Loss: 0.14
Loss: 0.03875
Loss: 0.033749999999999995


/home/ralampay/workspace/pattern-recognition-course/env/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


Loss: 0.03875


/home/ralampay/workspace/pattern-recognition-course/env/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


Loss: 0.03791666666666666
Loss: 0.03583333333333333
Loss: 0.03916666666666666
Loss: 0.03583333333333333


In [11]:
x_samples

[[0.0039000000000000003, 0.008400000000000001],
 [0.008400000000000001, 0.0064],
 [0.006500000000000001, 0.0001],
 [0.008, 0.0019],
 [0.001, 0.0079],
 [0.0007, 0.0011],
 [0.0015, 0.002],
 [0.0011, 0.008700000000000001],
 [0.0061, 0.0024000000000000002],
 [0.007500000000000001, 0.0026000000000000003]]

In [12]:
y_samples

[[0.04375],
 [0.04791666666666666],
 [0.14],
 [0.03875],
 [0.033749999999999995],
 [0.03875],
 [0.03791666666666666],
 [0.03583333333333333],
 [0.03916666666666666],
 [0.03583333333333333]]

In [13]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF

kernel = 1 * RBF(length_scale=1.0, length_scale_bounds=(1e-2, 1e2))
gaussian_process = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=9)
gaussian_process.fit(x_samples, y_samples)

mean, std = gaussian_process.predict(x_samples, return_std=True)

/home/ralampay/workspace/pattern-recognition-course/env/lib/python3.10/site-packages/sklearn/gaussian_process/_gpr.py:616: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
/home/ralampay/workspace/pattern-recognition-course/env/lib/python3.10/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified lower bound 0.01. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


In [14]:
mean

array([0.04375   , 0.04791667, 0.13999999, 0.03875005, 0.03375001,
       0.03875001, 0.03791665, 0.03583333, 0.03916671, 0.03583326])

In [15]:
std.reshape(-1, 1)

array([[9.99986719e-06],
       [1.00000004e-05],
       [1.00000004e-05],
       [1.00000448e-05],
       [1.00000004e-05],
       [1.00000004e-05],
       [1.00000004e-05],
       [9.99991160e-06],
       [1.00000004e-05],
       [1.00000892e-05]])

In [16]:
def f(params):
    model = MLPClassifier(
        random_state=1, 
        max_iter=1000,
        alpha=params['alpha'],
        learning_rate_init=params['learning_rate']
    )
    
    model.fit(x_train.values, y_train.values)

    predictions = model.predict(x_test.values)

    score = mean_squared_error(predictions, y_test.values)
    
    return score

In [17]:
from hyperopt import hp, fmin, tpe

params_list = {
    'alpha': hp.uniform('alpha', 0.001, 0.01),
    'learning_rate': hp.uniform('learning_rate', 0.0001, 0.001)
}

best_params = fmin(fn=f, space=params_list, max_evals=24, algo=tpe.suggest)

best_params

  4%|█████▊                                                                                                                                      | 1/24 [00:16<06:22, 16.63s/trial, best loss: 0.03583333333333333]

/home/ralampay/workspace/pattern-recognition-course/env/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(



  8%|███████████▋                                                                                                                                | 2/24 [00:27<04:56, 13.46s/trial, best loss: 0.03583333333333333]

/home/ralampay/workspace/pattern-recognition-course/env/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(



 12%|█████████████████▌                                                                                                                          | 3/24 [00:39<04:21, 12.47s/trial, best loss: 0.03583333333333333]

...

 96%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋      | 23/24 [04:33<00:11, 11.53s/trial, best loss: 0.0325]

/home/ralampay/workspace/pattern-recognition-course/env/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(



100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 24/24 [04:51<00:00, 12.15s/trial, best loss: 0.0325]


/home/ralampay/workspace/pattern-recognition-course/env/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(



{'alpha': 0.007949368103954593, 'learning_rate': 0.0009188281874523864}